In [1]:
import json
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')  # Ignore everything
# ignore everything does not work: ignore specific messages, using regex
warnings.filterwarnings('ignore', '.*A value is trying to be set on a copy of a slice from a DataFrame.*')
warnings.filterwarnings('ignore', '.*indexing past lexsort depth may impact performance*')

In [4]:
with open("../result/Gym_run0.json", "r") as read_file:
    data_run0 = json.load(read_file)

with open("../result/Gym_run1.json", "r") as read_file:
    data_run1 = json.load(read_file)

In [5]:
final_result = {}
for id in list(data_run0.keys()):
    ensemble_result = []
    ensemble_result = data_run0[id] + data_run1[id]
    df_new = pd.DataFrame(ensemble_result).groupby('answer')
    
    ensemble_df = pd.DataFrame()
    for i, row in df_new:
        if len(row) == 1 :
            ensemble_df = pd.concat([ensemble_df, row], axis=0)
        else:
            new_row = row.iloc[0]
            new_row['score'] = sum(row['score'])
            ensemble_df = pd.concat([ensemble_df, pd.DataFrame(new_row).transpose()], axis=0)


    ensemble_df = ensemble_df.sort_values(by=['score'], ascending=False)[:10]
    ensemble_df['rank'] = [i for i in range(1, len(ensemble_df)+1)]
    final_result[id] = list(ensemble_df.transpose().to_dict().values())


    

In [7]:
output_file = '../result/Gym_test_ensemble.json'
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(final_result, outfile, indent=2, ensure_ascii=False, default=int)